In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.head(3)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0
2,ce46411b1526c94f20a383b8cb188f8d27f82a0a,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,NO,32,All-net 500F=2000F;5d,3.0,0


In [4]:
train.columns

Index(['user_id', 'REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'ZONE1', 'ZONE2', 'MRG', 'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK',
       'CHURN'],
      dtype='object')

In [5]:
train['MRG'] = train['MRG'].factorize()[0]
train['TENURE'] = train['TENURE'].factorize()[0]


In [6]:
test['MRG'] = test['MRG'].factorize()[0]
test['TENURE'] = test['TENURE'].factorize()[0]

In [7]:
train = train.drop('TOP_PACK', axis=1)
test = test.drop('TOP_PACK', axis=1)

In [8]:
to_drop = [columns for columns in train.columns[train.isnull().mean()>=0.7]]
to_drop.extend([columns for columns in train.columns if columns not in ['user_id', 'REGION']
                if train[columns].std() == 0])
to_drop.append('user_id')
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)
print("%f columns were dropped"%(len(to_drop)))

4.000000 columns were dropped


In [9]:
train = train.fillna(-999)
test = test.fillna(-999)

In [10]:
train.head(2)

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,FREQ_TOP_PACK,CHURN
0,SAINT-LOUIS,0,17000.0,32.0,18000.0,6000.0,34.0,-999.0,97.0,355.0,6.0,62,35.0,0
1,-999,0,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,40,22.0,0


In [11]:
print(train['REGION'].value_counts())

-999           157520
DAKAR           95517
THIES           33374
SAINT-LOUIS     22125
LOUGA           18513
KAOLACK         18042
DIOURBEL        12576
TAMBACOUNDA     10273
KAFFRINE         8181
KOLDA            7204
FATICK           6638
MATAM            5255
ZIGUINCHOR       4012
SEDHIOU           589
KEDOUGOU          181
Name: REGION, dtype: int64


In [12]:
label = train['REGION'].astype('category').cat.categories.tolist()
replace = {'REGION' : {k: v for k,v in zip(label,list(range(1,len(label)+1)))}}
print(replace)

{'REGION': {-999: 1, 'DAKAR': 2, 'DIOURBEL': 3, 'FATICK': 4, 'KAFFRINE': 5, 'KAOLACK': 6, 'KEDOUGOU': 7, 'KOLDA': 8, 'LOUGA': 9, 'MATAM': 10, 'SAINT-LOUIS': 11, 'SEDHIOU': 12, 'TAMBACOUNDA': 13, 'THIES': 14, 'ZIGUINCHOR': 15}}


In [13]:
train.replace(replace, inplace=True) 

In [14]:
label = test['REGION'].astype('category').cat.categories.tolist()
replace = {'REGION' : {k: v for k,v in zip(label,list(range(1,len(label)+1)))}}
print(replace)
test.replace(replace, inplace=True)

{'REGION': {-999: 1, 'DAKAR': 2, 'DIOURBEL': 3, 'FATICK': 4, 'KAFFRINE': 5, 'KAOLACK': 6, 'KEDOUGOU': 7, 'KOLDA': 8, 'LOUGA': 9, 'MATAM': 10, 'SAINT-LOUIS': 11, 'SEDHIOU': 12, 'TAMBACOUNDA': 13, 'THIES': 14, 'ZIGUINCHOR': 15}}


train['FREQUENCY_DIFFERENCE'] = train['FREQUENCE_RECH'] - train['FREQUENCE']
#train['FREQUENCY_PER_DATA_VOLUME'] = train['DATA_VOLUME'] / train['FREQUENCE']
test['FREQUENCY_DIFFERENCE'] = test['FREQUENCE_RECH'] - test['FREQUENCE']
#test['FREQUENCY_PER_DATA_VOLUME'] = test['DATA_VOLUME'] / test['FREQUENCE']
#train['REGULARITY_PER_REVENUE'] = train['REVENUE'] / train['REGULARITY']
#test['REGULARITY_PER_REVENUE'] = test['REVENUE'] / test['REGULARITY']

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_trans = sc.fit_transform(train.drop(['CHURN'], axis = 1))
test_trans = sc.transform(test)

In [16]:
y= train['CHURN']
X = train_trans

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(320000, 13)
(80000, 13)
(320000,)
(80000,)


In [18]:
def loss(true, pred):
    from sklearn.metrics import log_loss
    return np.sqrt(log_loss(true, pred))

from sklearn.ensemble import GradientBoostingClassifier
gbr =GradientBoostingClassifier(n_estimators=900, learning_rate=0.07, max_depth=11)
gbr.fit(X_train, y_train)
y_pred = gbr.predict_proba(X_test)
loss(y_test, y_pred)

from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=450,random_strength=11,depth=8,learning_rate=0.05)
cat.fit(X_train, y_train)
y_pred = cat.predict_proba(X_test)
loss(y_test, y_pred)

In [19]:
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
kfold, scores = KFold(n_splits=4, shuffle=True, random_state=221), list()
pred_tot_cb = []
for train, test in kfold.split(X):
    x_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    model = CatBoostClassifier(random_state=42,  n_estimators=500, verbose=1000,learning_rate=0.07,
                               eval_metric='Logloss', early_stopping_rounds=250)
    model.fit(x_train, y_train)
    preds = model.predict_proba(x_test)
    score = log_loss(y_test, preds)
    scores.append(score)
    test_pred = model.predict(test_trans)
    pred_tot_cb.append(test_pred)
    print(score)
print("Average: ", sum(scores)/len(scores))

0:	learn: 0.5819468	total: 662ms	remaining: 5m 30s
499:	learn: 0.2471125	total: 1m 39s	remaining: 0us
0.25154625822460747
0:	learn: 0.5883751	total: 194ms	remaining: 1m 36s
499:	learn: 0.2467036	total: 1m 35s	remaining: 0us
0.25316249693277365
0:	learn: 0.5888112	total: 220ms	remaining: 1m 49s
499:	learn: 0.2465459	total: 1m 41s	remaining: 0us
0.25389201751353646
0:	learn: 0.5819622	total: 186ms	remaining: 1m 32s
499:	learn: 0.2477039	total: 1m 41s	remaining: 0us
0.2507624880085633
Average:  0.2523408151698702


In [20]:
pred_tot_cb

[array([1, 1, 0, ..., 0, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 0], dtype=int64)]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

n_estimators=[500,800,1500,]
max_features=['auto', 'sqrt', 'log2']
max_depth=[15,30,50]
max_depth.append(None)
min_samples_split=[5,15,20]
min_samples_leaf=[5,10,15]

grid_param = {'n_estimators': n_estimators,
             'max_features': max_features,
             'max_depth': max_depth,
             'min_samples_split': min_samples_split,
             'min_samples_leaf': min_samples_leaf}

rfr=RandomForestClassifier(random_state=1)
rfr_random=RandomizedSearchCV(estimator = rfr,
                             param_distributions = grid_param, n_iter = 2,
                             cv = 4, verbose=2, random_state=42,
                             n_jobs = -1)
rfr_random.fit(X_train, y_train)
print(rfr_random.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators=500, min_samples_split=15, min_samples_leaf=15, max_features='auto', max_depth=30)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)
loss(y_test, y_pred)

from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=1000,random_strength=9,depth=7,learning_rate=0.05)
#random_strength': 9, 'learning_rate': 0.05, 'iterations': 900, 'depth': 7
cat.fit(X_train, y_train)
y_pred = cat.predict_proba(X_test)
loss(y_test, y_pred)

In [21]:
prediction = model.predict_proba(test_trans)[:,1]

In [22]:
print(prediction[:1])

[0.78326352]


In [23]:
submit = pd.read_csv('sample_submission.csv')
submit.head()

,user_id,CHURN
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,0
1,5335efd940280b82143272275637d1e65d37eadb,0
2,a581f4fa08677c26f83f643248c667e241043086,0
3,64f67177d0775262b8087a9e2e3b8061b6324ae6,0
4,0d6009a4594c4be22449b8d9cc01a0bcea98faea,0


In [24]:
pd.DataFrame({'user_id':submit.user_id,'CHURN':prediction}).to_csv('Acesubmission.csv', index=False)
print('Donee')

Donee
